In [17]:
import sys
sys.path.append('/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/')
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path
import yfinance as yf
import time
from datetime import timedelta, datetime
from systems.performance_reporter import StrategyReporter, PerformanceReporter
from systems.utils import MarketDataExtractor

In [18]:
market_data_extractor = MarketDataExtractor()
strategy_reporter = StrategyReporter()
performance_reporter = PerformanceReporter(market_data_extractor)

In [19]:
# All the Inputs for the Backtest that needs to be analyzed
testname = '2d141f153488d338f3ba63ae46f9d97fb9318eb51b7f84952a460287793b7722_8' # Profits not Reinvested
# testname = '3aa6bc1bc58ab86f2a23419d3737ec62991783e80d8d8802f5cc68a1cfb0a33b' # Profits Reinvested & Margin Deployed @ 0.65 of 3.3x
save_folder = "/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/db/data/yahoo/1d"
test_folder_path = f'/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/db/vault/backtest_reports/{testname}'
backtest_output = strategy_reporter.load_backtest_output(test_folder_path)
open_orders = backtest_output['open_orders']
closed_orders = backtest_output['closed_orders']
config_dict = backtest_output['config_dict']

In [20]:
#Processing Open order to make them close, and remove history
open_orders_list = strategy_reporter.clean_up_order_list(open_orders, drop_history=True)
#Processing Closed orders to remove history
closed_orders_list = strategy_reporter.clean_up_order_list(closed_orders, drop_history=True)
#Merge both orders
final_list = closed_orders_list + open_orders_list
# Step 2: Sort the filtered orders by exit order `filled_timestamp`
sorted_orders = sorted(final_list, key=lambda x: x[1]['filled_timestamp'])
# Step 3: Fetch the first and last `filled_timestamp` with buffer
start_date = sorted_orders[0][1]['filled_timestamp'] - timedelta(days=365)
end_date = sorted_orders[-1][1]['filled_timestamp'] + timedelta(days=365)
# end_date = pd.Timestamp(datetime(2010, 1, 1))
# Current we are downloading csv file for below three index
indices = ["^DJI", "^IXIC", "^GSPC"]

index_data_dict = strategy_reporter.load_and_save_index_data(indices, start_date, end_date, save_folder)
rolling_window = int(max(60, (end_date - start_date).days / 20))
print({f"Analyzing with a rolling window of {rolling_window} days"})

{'Analyzing with a rolling window of 75 days'}


### This below function calculates profit/loss from the orders  **(exitPrice - entryPrice) * orderQuantity**

This function also plot the progress lines for 3 indexes provided earlier.

In [21]:
starting_capital = 72267.75
trade_df_by_date = strategy_reporter.create_data_input_for_cumulative_returns_and_indices(final_list, index_data_dict, starting_capital)
strategy_reporter.plot_cumulative_returns_and_indices(trade_df_by_date, index_data_dict)

2025-01-22 17:33:50,169 - REPORTER - INFO - Starting trade data preparation... - performance_reporter.py:26
2025-01-22 17:33:50,177 - REPORTER - ERROR - Account value went negative, invalid trading scenario - performance_reporter.py:92


AttributeError: 'NoneType' object has no attribute 'index'

In [7]:
sorted_orders[0]

[{'symbol': 'BAC',
  'timestamp': Timestamp('2009-01-09 23:59:59+0000', tz='UTC'),
  'orderDirection': 'BUY',
  'entryPrice': 13.24,
  'orderType': 'MARKET',
  'timeInForce': 'DAY',
  'orderQuantity': 1323.0,
  'strategy_name': 'pairs_trading',
  'broker': 'SIM',
  'granularity': '1d',
  'status': 'closed',
  'signal_id': '1fc462021beda3be3201895bc934b9c5a12901717cb9ab4aba4020fc7ebba99c',
  'symbol_ltp': {'2009-01-09 23:59:59+00:00': 13.24},
  'strategy_inputs': {'strategy_order_id': 'BAC_JPM_66a7c9990d58412cb5c9249743108f16'},
  'fill_price': 13.24,
  'filled_timestamp': Timestamp('2009-01-09 23:59:59+0000', tz='UTC'),
  'order_id': 'eeb1afe4b61642d8516b4752fab9fd589e3517a7a3d320b61c454ffda93df7c8',
  'broker_order_id': 'eeb1afe4b61642d8516b4752fab9fd589e3517a7a3d320b61c454ffda93df7c8',
  'message': 'Order filled at market price.',
  'fresh_update': False},
 {'symbol': 'BAC',
  'timestamp': Timestamp('2009-01-09 23:59:59+0000', tz='UTC'),
  'orderDirection': 'SELL',
  'exitPrice': 14.

# Plot Rolling charts

In [8]:
pair_trading_orders_dict = {}

for order_history in closed_orders:
    if order_history[0]['strategy_inputs']['strategy_order_id'] not in pair_trading_orders_dict.keys():
        pair_trading_orders_dict[order_history[0]['strategy_inputs']['strategy_order_id']] = []
    pair_trading_orders_dict[order_history[0]['strategy_inputs']['strategy_order_id']].append(
        {'symbol':order_history[0]['symbol'], 
         'direction':order_history[0]['orderDirection'],
         'entry_price':order_history[0]['fill_price'], 
         'exit_price':order_history[-1]['fill_price'], 
         'order_quantity':order_history[0]['orderQuantity'],
         'entry_timestamp':order_history[0]['filled_timestamp'], 
         'symbol_ltp-len':len(order_history[-1]['symbol_ltp']),
         'exit_timestamp':order_history[-1]['filled_timestamp'], 
         'profit': ((order_history[-1]['fill_price'] - order_history[0]['fill_price']) * (1 if order_history[0]['orderDirection'] == 'BUY' else -1) * (order_history[0]['orderQuantity']))
         })

In [9]:
pair_trading_orders_dict

{'BAC_JPM_66a7c9990d58412cb5c9249743108f16': [{'symbol': 'BAC',
   'direction': 'BUY',
   'entry_price': 13.24,
   'exit_price': 14.053549482287726,
   'order_quantity': 1323.0,
   'entry_timestamp': Timestamp('2009-01-09 23:59:59+0000', tz='UTC'),
   'symbol_ltp-len': 6,
   'exit_timestamp': Timestamp('2009-01-15 23:59:59+0000', tz='UTC'),
   'profit': 1076.3259650666619},
  {'symbol': 'JPM',
   'direction': 'SELL',
   'entry_price': 31.01,
   'exit_price': 29.814283157228477,
   'order_quantity': 913.0,
   'entry_timestamp': Timestamp('2009-01-09 23:59:59+0000', tz='UTC'),
   'symbol_ltp-len': 7,
   'exit_timestamp': Timestamp('2009-01-16 23:59:59+0000', tz='UTC'),
   'profit': 1091.689477450402}],
 'XOM_CVX_04b152c5a54f43a990169293ad307aff': [{'symbol': 'CVX',
   'direction': 'BUY',
   'entry_price': 73.97,
   'exit_price': 75.14601005992478,
   'order_quantity': 718.0,
   'entry_timestamp': Timestamp('2009-01-10 23:59:59+0000', tz='UTC'),
   'symbol_ltp-len': 7,
   'exit_timestamp'

In [10]:
pair_trading_orders_dict_ = {}
count = 0
for key, value in pair_trading_orders_dict.items():
    if len(value) == 2:
        # print(len(value))
        pair_trading_orders_dict_[key] = value
pair_trading_orders_dict = pair_trading_orders_dict_

In [11]:
both_profit_count = 0
both_loss_count = 0
mixed_count = 0

for key, value in pair_trading_orders_dict.items():
    order_1, order_2 = value
    order_1_profit = order_1['profit']
    order_2_profit = order_2['profit']
    if order_1_profit > 0 and order_2_profit > 0:
        both_profit_count += 1
    elif order_1_profit < 0 and order_2_profit < 0:
        both_loss_count += 1
    else:
        mixed_count += 1
        
    print(f"Order 1 PnL: order_1: {order_1_profit}, Order 2 PnL: {order_2_profit}, Total PnL: {order_1['profit'] + order_2['profit']}")

Order 1 PnL: order_1: 1076.3259650666619, Order 2 PnL: 1091.689477450402, Total PnL: 2168.015442517064
Order 1 PnL: order_1: 844.3752230259904, Order 2 PnL: 3418.9596543046346, Total PnL: 4263.334877330625
Order 1 PnL: order_1: -391.31999999999937, Order 2 PnL: 1276.6543818878324, Total PnL: 885.334381887833
Order 1 PnL: order_1: -204.81999999999755, Order 2 PnL: 1186.4752217333046, Total PnL: 981.655221733307
Order 1 PnL: order_1: -119.34999999999938, Order 2 PnL: 1149.8043275730436, Total PnL: 1030.454327573044
Order 1 PnL: order_1: -400.70999999999725, Order 2 PnL: 668.816711286991, Total PnL: 268.10671128699374
Order 1 PnL: order_1: -385.9199999999994, Order 2 PnL: 5357.122073711108, Total PnL: 4971.202073711109
Order 1 PnL: order_1: -58.200000000000486, Order 2 PnL: 2703.3593259785293, Total PnL: 2645.159325978529
Order 1 PnL: order_1: -159.47999999999973, Order 2 PnL: 2298.080300443193, Total PnL: 2138.6003004431936
Order 1 PnL: order_1: -155.84000000000015, Order 2 PnL: -491.049

In [12]:
both_profit_count, both_loss_count, mixed_count

(39, 56, 354)

In [13]:
pair_trading_orders_dict

{'BAC_JPM_66a7c9990d58412cb5c9249743108f16': [{'symbol': 'BAC',
   'direction': 'BUY',
   'entry_price': 13.24,
   'exit_price': 14.053549482287726,
   'order_quantity': 1323.0,
   'entry_timestamp': Timestamp('2009-01-09 23:59:59+0000', tz='UTC'),
   'symbol_ltp-len': 6,
   'exit_timestamp': Timestamp('2009-01-15 23:59:59+0000', tz='UTC'),
   'profit': 1076.3259650666619},
  {'symbol': 'JPM',
   'direction': 'SELL',
   'entry_price': 31.01,
   'exit_price': 29.814283157228477,
   'order_quantity': 913.0,
   'entry_timestamp': Timestamp('2009-01-09 23:59:59+0000', tz='UTC'),
   'symbol_ltp-len': 7,
   'exit_timestamp': Timestamp('2009-01-16 23:59:59+0000', tz='UTC'),
   'profit': 1091.689477450402}],
 'XOM_CVX_04b152c5a54f43a990169293ad307aff': [{'symbol': 'CVX',
   'direction': 'BUY',
   'entry_price': 73.97,
   'exit_price': 75.14601005992478,
   'order_quantity': 718.0,
   'entry_timestamp': Timestamp('2009-01-10 23:59:59+0000', tz='UTC'),
   'symbol_ltp-len': 7,
   'exit_timestamp'

In [16]:
closed_orders[0]

[{'symbol': 'BAC',
  'timestamp': Timestamp('2009-01-09 23:59:59+0000', tz='UTC'),
  'orderDirection': 'BUY',
  'entryPrice': 13.24,
  'orderType': 'MARKET',
  'timeInForce': 'DAY',
  'orderQuantity': 1323.0,
  'strategy_name': 'pairs_trading',
  'broker': 'SIM',
  'granularity': '1d',
  'status': 'closed',
  'signal_id': '1fc462021beda3be3201895bc934b9c5a12901717cb9ab4aba4020fc7ebba99c',
  'symbol_ltp': {'2009-01-09 23:59:59+00:00': 13.24},
  'strategy_inputs': {'strategy_order_id': 'BAC_JPM_66a7c9990d58412cb5c9249743108f16'},
  'fill_price': 13.24,
  'filled_timestamp': Timestamp('2009-01-09 23:59:59+0000', tz='UTC'),
  'order_id': 'eeb1afe4b61642d8516b4752fab9fd589e3517a7a3d320b61c454ffda93df7c8',
  'broker_order_id': 'eeb1afe4b61642d8516b4752fab9fd589e3517a7a3d320b61c454ffda93df7c8',
  'message': 'Order filled at market price.',
  'fresh_update': False},
 {'symbol': 'BAC',
  'timestamp': Timestamp('2009-01-09 23:59:59+0000', tz='UTC'),
  'orderDirection': 'SELL',
  'exitPrice': 14.